In [191]:
import numpy as np
import pandas as pd
import csv
import sys

In [192]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
Y_test = pd.read_csv('data/gender_submission.csv')

In [193]:
X_train = train[['Pclass', 'Fare']]
X_test = test[['Pclass', 'Fare']]
Y_train = train['Survived']

In [194]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [195]:
class GenerativeModel(object):
    def __init__(self):
        a = None
        self.con_inv = None
        
    def fit(self, X, y):
        # Find meand, coveriance matrix of two class
        self.c1_ind = np.where(y==0)[0]
        self.c2_ind = np.where(y==1)[0]
        
        self.c1_X = np.array(X)[self.c1_ind, :]
        self.c2_X = np.array(X)[self.c2_ind, :]
        self.u1 = np.mean(self.c1_X, axis=0)
        self.u2 = np.mean(self.c2_X, axis=0)
        self.p1 = len(self.c1_ind) / (len(self.c1_ind) + len(self.c2_ind))
        self.p2 = 1 - self.p1
        self.cov =  np.cov(self.c1_X,rowvar=False) * self.p1 + np.cov(self.c2_X,rowvar=False) * self.p2
#         print(self.cov)
        self.cov_inv = np.linalg.pinv(self.cov)
        
    def predict(self, test_X):
        test_X = np.array(test_X)
        z =((test_X).dot(self.cov_inv).dot(self.u1-self.u2) - \
            (1/2)*(self.u1).dot(self.cov_inv).dot(self.u1)+ (1/2)\
            *(self.u2).dot(self.cov_inv).dot(self.u2)+\
            np.log(len(self.c1_ind)/len(self.c2_ind)))
        return self.sign(self.sigmoid(z))
    
    def sigmoid(self, z):
        res = 1/(1+np.exp(-z))
        return np.clip(res, 1e-10, 1-(1e-10))
    
    def sign(self, a):
        output = []
        for i in a:
            if i > 0.5:
                output.append(1)
            else:
                output.append(0)
        return output

In [196]:
GM = GenerativeModel()

In [197]:
GM.fit(X_train, Y_train)
pred_y = GM.predict(X_test)

In [198]:
len(Y_test) - len(np.where(Y_test==0)[0])

152

In [199]:
len(np.where((Y_test['Survived'] == pred_y) ==True )[0]) / len(Y_test)

0.3827751196172249